In [1]:
import matplotlib.pyplot as plt
import numpy             as np
import pandas            as pd
import seaborn           as sns
import os
import torch
import json
import sys

from pymatgen.core.structure         import Structure
from pymatgen.symmetry.bandstructure import HighSymmKpath

from libraries.model   import Helmholtz_free_energy_function, make_predictions, GCNN, compute_coefficients, compute_Fv
from libraries.dataset import load_atomic_masses, include_temperatures, create_predictions_dataset, standardize_dataset_from_keys

sys.path.append('../../UPC')
import MP.MP_library       as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sns.set_theme()

In [2]:
model_folder    = 'model'  # Pre-trained model and dataset parameters
output_folder   = 'output'  # Output files and figures
input_folder    = 'input'  # General files (e.g., atomic masses information)

# Whether to plot the harmonic extrapolations of Fv (very time-consuming) or not
plot_extrapolations = False

# Defining the range of temperatures
Ti = 300
Tf = 600
dT = 5
temperatures = np.arange(Ti, Tf+dT, dT)  # Temperatures for prediction of free-energies

# Loading dictionary of atomic masses
atomic_masses = load_atomic_masses(f'{input_folder}/atomic_masses.dat')

In [3]:
# Load the data from the JSON file
with open(f'{model_folder}/standardized_parameters.json', 'r') as json_file:
    numpy_dict = json.load(json_file)

# Convert NumPy arrays back to PyTorch tensors
standardized_parameters = {}
for key, value in numpy_dict.items():
    try:
        standardized_parameters[key] = torch.tensor(value)
    except:
        standardized_parameters[key] = value

# Load reference dataset for uncertainty estimation
reference_dataset = torch.load(f'{model_folder}/ref_std_dataset.pt', weights_only=False)

In [4]:
colors = ['b', 'r', 'g', 'k']

materials = {
    'CO': ['P2_12_12_1', 'R3c'],
    'MgTe': ['F-43m', 'P6_3mc'],
    'SeN': ['C2-c', 'P2_1-c'],
    'SrC2': ['C2-c', 'I4-mmm'],
    'WCl6': ['P-3m1', 'R-3'],
    'WN2': ['P3_121', 'Pna2_1'],
    'WSe2': ['P6_3-mmc', 'P-6m2'],
    'ZnCl2': ['P2_1-c', 'P4_2-nmc'],
    'ZnTe': ['F-43m', 'P6_3mc'],
    'ZrSeO': ['P2_13', 'P4-nmm']
}

In [5]:
def pred_ML_Fv(path_to_POSCAR, temperatures, standardized_parameters, model_folder, device):
    # Create dataset for predictions
    dataset = create_predictions_dataset(path_to_POSCAR, path_to_material=True, path_to_polymorph=True)
    
    labels = [graph.label for graph in dataset]

    # Standardize properties
    std_dataset = standardize_dataset_from_keys(dataset, standardized_parameters)

    # Load Graph Neural Network model (making room for temperature as node attribute) to device
    # Dropout for initializing the model, not used at all while predicting
    model = GCNN(features_channels=dataset[0].num_node_features+1,
                 pdropout=0).to(device)

    # Free-up CUDA
    del dataset

    # Load and evaluate Graph Neural Network model
    model.load_state_dict(torch.load(f'{model_folder}/model.pt', map_location=torch.device(device)))
    model.eval()

    # Include temperatures
    std_dataset_w_temp = include_temperatures(std_dataset, temperatures, standardized_parameters)

    # Free-up CUDA
    del std_dataset

    # Compute predictions and corresponding uncertainties
    shot_predictions, shot_uncertainties = make_predictions(reference_dataset, std_dataset_w_temp, model, standardized_parameters)

    # Free-up CUDA
    del std_dataset_w_temp

    # Computing the coefficients and uncertainties from fitting
    coefficients = compute_coefficients(temperatures, shot_predictions, shot_uncertainties, s=0.5)
    
    # Compute Fv
    Fv_pred = compute_Fv(temperatures, coefficients)[0]
    return  Fv_pred, shot_uncertainties

def pred_DFT_Fv(path_to_PHONON, temperatures):
    # Loading number of atoms
    _, _, concentration, _ = MPL.information_from_VASPfile(path_to_PHONON, file='POSCAR')
    n_atoms = np.sum(concentration)
    
    # Reading supercell information
    dim_info = MPL.read_phonopyconf(path_to_PHONON)
    
    # Write mesh.conf file (needed for phonopy)
    MPL.write_meshconf(path_to_PHONON, material, dim_info, Ti, Tf, dT)
    
    # Getting thermal properties with phonopy (ignoring output)
    previous_dir = os.getcwd()
    os.chdir(path_to_PHONON)
    os.system('phonopy -t mesh.conf > /dev/null')
    os.chdir(previous_dir)
    
    # Read generated thermal properties (kJ/mol)
    _, Fv_PHONON = MPL.read_thermalpropertyyaml(len(temperatures), path_to_PHONON, thermalproperty='free_energy')
    
    # Pass kJ / molmp-1009220 to meV / atom
    conversion_factor = 1.6 * 6.022 * 0.01 * n_atoms
    Fv_PHONON        /= conversion_factor
    return Fv_PHONON

In [6]:
uncert_DFT_0 = np.ones(len(temperatures)) * 4
uncert_DFT_1 = np.ones(len(temperatures)) * 4

uplims = [False] * len(temperatures)
lolims = [False] * len(temperatures)

for material in materials.keys():
    print(material)

    polymorphs = materials[material]
    path_to_material = f'/home/claudio/Desktop/validation/{material}'

    # IDX = 0
    idx = 0
    path_to_polymorph = f'{path_to_material}/{polymorphs[idx]}'
    path_to_MP = f'{path_to_polymorph}/relaxation'
    path_to_phonons = f'{path_to_polymorph}/phonons'

    if not os.path.exists(path_to_MP):
        continue
    
    Fv_ML_MP_0, uncert_ML_MP_0 = pred_ML_Fv(path_to_MP,
                                            temperatures, standardized_parameters, model_folder, device)
    Fv_ML_phonons_0, uncert_ML_phonons_0 = pred_ML_Fv(path_to_phonons,
                                                      temperatures, standardized_parameters, model_folder, device)
    Fv_DFT_0 = pred_DFT_Fv(path_to_phonons,
                           temperatures)
    
    epa_0 = float(np.loadtxt(f'{path_to_MP}/EPA')) * 1e3  # From eV/atom to meV/atom

    
    # IDX = 1
    idx = 1
    path_to_polymorph = f'{path_to_material}/{polymorphs[idx]}'
    path_to_MP = f'{path_to_polymorph}/relaxation'
    path_to_phonons = f'{path_to_polymorph}/phonons'

    if not os.path.exists(path_to_MP):
        continue
    
    Fv_ML_MP_1, uncert_ML_MP_1 = pred_ML_Fv(path_to_MP,
                                            temperatures, standardized_parameters, model_folder, device)
    Fv_ML_phonons_1, uncert_ML_phonons_1 = pred_ML_Fv(path_to_phonons,
                                                      temperatures, standardized_parameters, model_folder, device)
    Fv_DFT_1 = pred_DFT_Fv(path_to_phonons,
                           temperatures)
    
    epa_1 = float(np.loadtxt(f'{path_to_MP}/EPA')) * 1e3  # From eV/atom to meV/atom

    
    # Diff
    F_ML_MP_diff = (epa_1+Fv_ML_MP_1) - (epa_0+Fv_ML_MP_0)
    uncert_ML_MP_diff = np.sqrt(uncert_ML_MP_0**2 + uncert_ML_MP_1**2)

    F_ML_phonons_diff = (epa_1+Fv_ML_phonons_1) - (epa_0+Fv_ML_phonons_0)
    uncert_ML_phonons_diff = np.sqrt(uncert_ML_phonons_0**2 + uncert_ML_phonons_1**2)

    F_DFT_diff = (epa_1+Fv_DFT_1) - (epa_0+Fv_DFT_0)
    uncert_DFT_diff = np.sqrt(uncert_DFT_0**2 + uncert_DFT_1**2)
    
    # Plotting
    label = f'{polymorphs[1]}-{polymorphs[0]}'
    plt.errorbar(temperatures, F_ML_MP_diff,      color=colors[0], yerr=uncert_ML_MP_diff,      label='ML original', 
                 fmt=':o', uplims=uplims, lolims=lolims)
    plt.errorbar(temperatures, F_ML_phonons_diff, color=colors[1], yerr=uncert_ML_phonons_diff, label='ML relaxed',
                 fmt=':o', uplims=uplims, lolims=lolims)
    plt.errorbar(temperatures, F_DFT_diff,        color=colors[2], yerr=uncert_DFT_diff,        label='DFT',
                 fmt=':o', uplims=uplims, lolims=lolims)
        
    plt.xlabel(r'$T$ (K)')
    plt.ylabel(r'$\Delta F$ (meV/atom)')
    plt.legend(loc='best')
    plt.savefig(f'{path_to_material}/{material}-diff.png', dpi=300, bbox_inches='tight')
    plt.show()

CO


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
